# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [ ]:
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate

In [ ]:
seed =42
now = datetime.now().strftime("%Y%m%d")

In [ ]:
dataset = load_dataset("cais/mmlu", "all")

# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(20))

# user provided data description
DESCRIPTION = """
This is a massive multitask test consisting of multiple-choice questions from various branches of knowledge.
The test spans subjects in the humanities, social sciences, hard sciences, and other areas that are important for some people to learn.
To attain high accuracy on this test, models must possess extensive world knowledge and problem solving ability.
This covers 57 subjects  across STEM, the humanities, the social sciences, and more. 
It ranges in difficulty from an elementary level to an advanced professional level, and it tests both world knowledge and problem solving ability. 
Subjects range from traditional areas, such as mathematics and history, to more specialized areas like law and ethics.
"""


In [ ]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [ ]:
prompt = [gemini_prompt_template.format(description= DESCRIPTION,
                                        datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

In [ ]:
models = [
    # "palm",
    # "gemini",
    "claude"
    ]

ann = Annotate()


In [ ]:
r = await ann.classification(prompt, models=models)

In [ ]:
result = [dataset['choices'][idx][v] for idx, v in enumerate(dataset['answer'])]
pprint(result)

# debug GLAD

In [1]:
from utils import glad

In [2]:
tc = [20000, 20, 1000, 2, 0.5, 0.5]

In [3]:
import json

with open('./data/output/annotation_output__20240515.json', 'r') as file:
    ddddd = json.load(file)

In [4]:
glad(ddddd, tc)

{'alpha': {0: 1.8857004498502001,
  1: 1.8857004498502044,
  2: 1.0,
  3: 1.0,
  4: 1.0,
  5: 1.0,
  6: 1.0,
  7: 1.0,
  8: 1.0,
  9: 1.0,
  10: 1.0,
  11: 1.0,
  12: 1.0,
  13: 1.0,
  14: 1.0,
  15: 1.0,
  16: 1.0,
  17: 1.0,
  18: 1.0,
  19: 1.0},
 'beta': {0: 2.8532904907365597,
  1: 2.8532904907365597,
  2: 2.8532904907365597,
  3: 2.8532904907365597,
  4: 2.8532904907365597,
  5: 2.8532904907365597,
  6: 2.8532904907365597,
  7: 2.8532904907365597,
  8: 2.8532904907365597,
  9: 2.8532904907365597,
  10: 2.8532904907365597,
  11: 2.8532904907365597,
  12: 2.8532904907365597,
  13: 2.8532904907365597,
  14: 2.8532904907365597,
  15: 2.8532904907365597,
  16: 2.8532904907365597,
  17: 2.8532904907365597,
  18: 2.8532904907365597,
  19: 2.8532904907365597,
  20: 2.8532904907365597,
  21: 2.8532904907365597,
  22: 2.8532904907365597,
  23: 2.8532904907365597,
  24: 2.8532904907365597,
  25: 2.8532904907365597,
  26: 2.8532904907365597,
  27: 2.8532904907365597,
  28: 2.8532904907365597